In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import Control
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

from ribasim import Node
from ribasim.nodes import pump, level_boundary, tabulated_rating_curve
from shapely import Point

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "WetterskipFryslan"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -0.42  # default LevelBoundary level
default_level = -2.3456  # default LevelBoundary level


## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_8"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed
)
processor.run()

Swapped edge direction between Node A: 4069 and Node B: 3124
Swapped edge direction between Node A: 3124 and Node B: 360
Swapped edge direction between Node A: 4070 and Node B: 3125
Swapped edge direction between Node A: 3125 and Node B: 360
Swapped edge direction between Node A: 402 and Node B: 1740
Swapped edge direction between Node A: 1740 and Node B: 137
Swapped edge direction between Node A: 258 and Node B: 2164
Swapped edge direction between Node A: 2164 and Node B: 137
Swapped edge direction between Node A: 137 and Node B: 1819
Swapped edge direction between Node A: 1819 and Node B: 26
Processed all actions
The target levels (streefpeilen) have been updated.
The function of the pumps have been updated.


#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [6]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [7]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [8]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

#change high initial states to 0
ribasim_model.basin.state.df.loc[ribasim_model.basin.state.df["level"] == 9.999, "level"] = 0
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == 9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)


In [9]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

In [10]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(158166, 553915)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(158166, 553915)), [tabulated_rating_curve.Static(
            level=[0.0, 1.0],
            flow_rate=[0.0, 1.0])])

ribasim_model.edge.add(ribasim_model.basin[89], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [11]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(156824, 552856)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(156831, 552967)), [tabulated_rating_curve.Static(
            level=[0.0, 1.0],
            flow_rate=[0.0, 1.0])])

ribasim_model.edge.add(ribasim_model.basin[861], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [12]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(155989, 563067)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(156038, 563080)), [tabulated_rating_curve.Static(
            level=[0.0, 1.0],
            flow_rate=[0.0, 1.0])])

ribasim_model.edge.add(ribasim_model.basin[149], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [13]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(154647, 567409)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(154644, 567399)), [tabulated_rating_curve.Static(
            level=[0.0, 1.0],
            flow_rate=[0.0, 1.0])])

ribasim_model.edge.add(ribasim_model.basin[149], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [14]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(156173, 576551)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(157109, 576125)), [tabulated_rating_curve.Static(
            level=[0.0, 1.0],
            flow_rate=[0.0, 1.0])])

ribasim_model.edge.add(ribasim_model.basin[254], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [15]:
# #add a zeegemaal
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(170894, 591637)), [level_boundary.Static(level=[default_level])])

pump = ribasim_model.pump.add(Node(new_node_id+1, Point(170902, 591432)), [pump.Static(flow_rate=[700/60])])

ribasim_model.edge.add(ribasim_model.basin[4], pump)
ribasim_model.edge.add(pump, level_boundary_node)

In [16]:
# # #add a zeegemaal
# new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

# level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(170898, 591653)), [level_boundary.Static(level=[default_level])])

# pump = ribasim_model.pump.add(Node(new_node_id+1, Point(170903,591434)), [pump.Static(flow_rate=[700/60])])

# ribasim_model.edge.add(ribasim_model.basin[328], pump)
# ribasim_model.edge.add(pump, level_boundary_node)

In [17]:
# new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

# level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(156009, 563075)), [level_boundary.Static(level=[default_level])])

# tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(156025, 563076)), [tabulated_rating_curve.Static(
#             level=[0.0, 1.0],
#             flow_rate=[0.0, 1.0])])

# ribasim_model.edge.add(ribasim_model.basin[41], tabulated_rating_curve_node)
# ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [18]:
# new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

# level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(154644, 567398)), [level_boundary.Static(level=[default_level])])

# tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(154644, 567398)), [tabulated_rating_curve.Static(
#             level=[0.0, 1.0],
#             flow_rate=[0.0, 1.0])])

# ribasim_model.edge.add(ribasim_model.basin[41], tabulated_rating_curve_node)
# ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [19]:
# new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

# level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(156182, 576504)), [level_boundary.Static(level=[default_level])])

# tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(156624, 576519)), [tabulated_rating_curve.Static(
#             level=[0.0, 1.0],
#             flow_rate=[0.0, 1.0])])

# ribasim_model.edge.add(ribasim_model.basin[282], tabulated_rating_curve_node)
# ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [20]:
# ribasim_model.edge.df.loc[ribasim_model.edge.df.from_node_id==41, 'meta_from_node_type'] = 'Basin'
# ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id==3727, 'meta_to_node_type'] = 'TabulatedRatingCurve'
# ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id==3727, 'meta_categorie'] = 'hoofdwater'


# ribasim_model.edge.df.loc[ribasim_model.edge.df.from_node_id==3727, 'meta_from_node_type'] = 'TabulatedRatingCurve'
# ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id==3726, 'meta_to_node_type'] = 'LevelBoundary'
# ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id==3726, 'meta_categorie'] = 'hoofdwater'



In [21]:
ribasim_model.level_boundary.node.df.meta_node_id = ribasim_model.level_boundary.node.df.index
ribasim_model.tabulated_rating_curve.node.df.meta_node_id = ribasim_model.tabulated_rating_curve.node.df.index
ribasim_model.pump.node.df.meta_node_id = ribasim_model.pump.node.df.index



## Implement standard profile and a storage basin

In [22]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [23]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [24]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10*2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [25]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [26]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [27]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [28]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [29]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [30]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [31]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


### Manning Resistance

In [32]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [33]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 25
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [34]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

# Set numerical settings

In [35]:
# Write model output
ribasim_model.use_validation = True
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2024, 4, 1)
ribasim_model.solver.saveat = 3600 * 24 #increase saveat to save storage
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

In [36]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

# Write model

In [38]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/WetterskipFryslan/modellen/WetterskipFryslan_parametrized/results
The model of waterboard WetterskipFryslan has been uploaded to the goodcloud in the directory of boezemmodel!


In [39]:
df_edge = pd.read_feather(os.path.join(output_dir, "flow.arrow"))


In [40]:
df_edge.sort_values(by='flow_rate')

,time,edge_id,from_node_id,to_node_id,flow_rate
281163,2024-02-07,4700,3156,1103,-126.493121
278397,2024-02-07,1934,71,3156,-126.493121
323229,2024-02-13,1934,71,3156,-125.906403
325995,2024-02-13,4700,3156,1103,-125.906403
338173,2024-02-15,1934,71,3156,-125.756353
...,...,...,...,...,...
337913,2024-02-15,1674,71,2886,150.363439
278137,2024-02-07,1674,71,2886,151.263737
280900,2024-02-07,4437,2886,391,151.263737
276998,2024-02-07,535,391,1724,151.263818
